# Prepping airbnb new york's listing data for prediction

In [324]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'sklearn'

# Importing my data

Data was downloaded at airbnb's open portal, i chose Bordeaux data dated until the 15th June 2025

Gathering the columns i'll need onward

In [ ]:
df_airbnb = pd.read_csv('Data/listings.csv')

wanted_cols = ['price','host_is_superhost','host_total_listings_count','neighbourhood_cleansed','property_type', 'room_type', 'accommodates', 'bathrooms','bedrooms', 'beds','availability_365','number_of_reviews']

df_houses = df_airbnb[wanted_cols].copy()

df_houses

,price,host_is_superhost,host_total_listings_count,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,availability_365,number_of_reviews
0,$266.00,t,31,Bordeaux Sud,Entire vacation home,Entire home/apt,4,1.0,2.0,3.0,248,127
1,$115.00,f,1,Saint-Mdard-en-Jalles,Entire rental unit,Entire home/apt,4,1.0,2.0,2.0,147,79
2,$203.00,t,31,Chartrons - Grand Parc - Jardin Public,Entire rental unit,Entire home/apt,3,1.5,1.0,2.0,273,80
3,$222.00,t,31,Centre ville (Bordeaux),Entire rental unit,Entire home/apt,6,1.0,2.0,4.0,269,161
4,$103.00,t,2,Centre ville (Bordeaux),Entire rental unit,Entire home/apt,2,1.0,0.0,1.0,353,556
...,...,...,...,...,...,...,...,...,...,...,...,...
12378,$65.00,f,3,Le Bouscat,Tiny home,Entire home/apt,2,1.0,1.0,1.0,360,0
12379,$117.00,f,19,Centre ville (Bordeaux),Entire home,Entire home/apt,4,1.0,2.0,2.0,44,0
12380,$63.00,f,3,Le Bouscat,Tiny home,Entire home/apt,1,1.0,1.0,1.0,365,0
12381,$35.00,f,2,Bordeaux Sud,Private room in rental unit,Private room,2,1.0,1.0,1.0,336,6


Checking column types and null counts

In [ ]:
print(df_houses.dtypes,'\n')
print(df_houses.isnull().sum())

price                         object
host_is_superhost             object
host_total_listings_count      int64
neighbourhood_cleansed        object
property_type                 object
room_type                     object
accommodates                   int64
bathrooms                    float64
bedrooms                     float64
beds                         float64
availability_365               int64
number_of_reviews              int64
dtype: object 

price                        4038
host_is_superhost             248
host_total_listings_count       0
neighbourhood_cleansed          0
property_type                   0
room_type                       0
accommodates                    0
bathrooms                    4008
bedrooms                     1052
beds                         4023
availability_365                0
number_of_reviews               0
dtype: int64


# Null drops and type casts

dropping rows with null price and casting the needed types

checking null counts and types after the changes

In [ ]:
df_houses.dropna(subset=['price'], inplace=True)

#type casting
df_houses['price'] =  df_houses['price'].str.replace('$','').str.replace(',','').astype(float)
df_houses['host_is_superhost'] = df_houses['host_is_superhost'].map({'t': 1, 'f': 0}).astype(bool)
df_houses['bedrooms'] = df_houses['bedrooms'].astype('Int64')
df_houses['beds'] = df_houses['beds'].astype('Int64')

print(df_houses.dtypes,'\n')
print(df_houses.isnull().sum())

price                        float64
host_is_superhost               bool
host_total_listings_count      int64
neighbourhood_cleansed        object
property_type                 object
room_type                     object
accommodates                   int64
bathrooms                    float64
bedrooms                       Int64
beds                           Int64
availability_365               int64
number_of_reviews              int64
dtype: object 

price                         0
host_is_superhost             0
host_total_listings_count     0
neighbourhood_cleansed        0
property_type                 0
room_type                     0
accommodates                  0
bathrooms                     3
bedrooms                     15
beds                          4
availability_365              0
number_of_reviews             0
dtype: int64


# Missing values % calc

Calculating the missing values % to see if imputing is viable

In [ ]:
(df_houses[['bathrooms', 'bedrooms', 'beds']].isnull().sum() / len(df)) * 100

bathrooms    0.035950
bedrooms     0.179748
beds         0.047933
dtype: float64

# Imputing missing values
every ratio was below 5% so i'll imput missing values using the cols mean

In [ ]:
df_houses['bathrooms'].fillna(df_houses['bathrooms'].median(),inplace=True)
df_houses['bedrooms'].fillna(df_houses['bedrooms'].median(),inplace=True)
df_houses['beds'].fillna(df_houses['beds'].median(),inplace=True)

print(df_houses.isnull().sum())

price                        0
host_is_superhost            0
host_total_listings_count    0
neighbourhood_cleansed       0
property_type                0
room_type                    0
accommodates                 0
bathrooms                    0
bedrooms                     0
beds                         0
availability_365             0
number_of_reviews            0
dtype: int64


# Checking Anormal values

In [ ]:
df_houses.describe()

,price,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,availability_365,number_of_reviews
count,8345.000000,8345.000000,8345.000000,8345.000000,8345.0,8345.0,8345.000000,8345.000000
mean,125.325824,15.959137,3.617376,1.279509,1.625884,2.040743,195.328580,46.403595
std,366.010877,131.519603,2.166303,0.636770,1.169893,1.458607,122.199195,94.762861
min,12.000000,1.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000
25%,52.000000,1.000000,2.000000,1.000000,1.0,1.0,81.000000,3.000000
50%,79.000000,2.000000,3.000000,1.000000,1.0,2.0,196.000000,14.000000
75%,130.000000,4.000000,4.000000,1.000000,2.0,3.0,314.000000,49.000000
max,10000.000000,8726.000000,16.000000,7.500000,9.0,24.0,365.000000,2764.000000


host_total_listings_count, checked the big values, related to agencys rather than people so ok no abnormal ! everything else seems ok !

# Standardization

Since the df will be use to guess prices, i'll standardize my values for better results !